In [1]:
#Importing all required packages
import matplotlib
import matplotlib.pyplot as plt 
%matplotlib inline
import pandas as pd 
import numpy as np
import seaborn as sns


import os
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, r2_score
import xgboost as xgb
from xgboost import XGBRegressor
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

np.random.seed(42)
import warnings 
warnings.filterwarnings('ignore')

In [2]:
#Loading Dataset with relative path
ed = pd.read_csv('energydata_complete.csv')
ed

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,...,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,...,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,...,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,...,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,...,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,10.084097,10.084097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,2016-05-27 17:20:00,100,0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,...,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,43.096812,43.096812
19731,2016-05-27 17:30:00,90,0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,...,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,49.282940,49.282940
19732,2016-05-27 17:40:00,270,10,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,...,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,29.199117,29.199117
19733,2016-05-27 17:50:00,420,10,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,...,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,6.322784,6.322784


In [3]:
#Data Pre-Processing
def myPreProc(ed):
    procData = ed.dropna(how = 'all', axis = 1, inplace = False)
    procData = procData.dropna() 
    procData['Power'] = procData['Appliances'].astype(int) + procData['lights'].astype(int)
    procData = procData.drop('Appliances', axis = 1);
    procData = procData.drop('lights', axis = 1);
    procData = procData.drop('rv1', axis = 1);
    procData = procData.drop('rv2', axis = 1);
    procData = procData.drop('date', axis = 1);
    return procData
procData = myPreProc(ed)
procData.isna().sum();
procData

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,...,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,Power
0,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,17.166667,55.200000,...,48.900000,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,90
1,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,17.166667,55.200000,...,48.863333,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,90
2,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,17.166667,55.090000,...,48.730000,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,80
3,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,17.166667,55.090000,...,48.590000,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,90
4,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,17.200000,55.090000,...,48.590000,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,23.200000,52.400000,...,50.074000,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,100
19731,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,23.230000,52.326667,...,49.790000,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,90
19732,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,23.230000,52.266667,...,49.660000,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,280
19733,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,23.200000,52.200000,...,49.518750,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333,430


In [4]:
#Splitting of Data
X = procData.drop('Power', axis=1)
y = y=procData['Power']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [5]:
estimators = [
    ('knn', KNeighborsRegressor()),
    ('xgb', XGBRegressor(random_state=42)),
    ('rf', RandomForestRegressor(random_state=42))
]

In [6]:
meta_model = RandomForestRegressor(random_state=42)

In [7]:
stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_model
)

In [8]:
stacking_regressor.fit(X_train, y_train)

StackingRegressor(estimators=[('knn', KNeighborsRegressor()),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interactio...
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=42, ...)),
                              ('rf', RandomForestRegressor(random_state=42))],
                  final_estimator=RandomForestRegressor(random_state=42))

In [9]:
y_train_pred = stacking_regressor.predict(X_train)
print('The training R_square is: %.2f'% r2_score(y_train, y_train_pred))
print('The MAE is:%2.f'%mean_absolute_error(y_train, y_train_pred))
print('MSE is: %2.f'%mean_squared_error(y_train, y_train_pred))
print('The RMSE is: %2.f'% np.sqrt(mean_squared_error(y_train, y_train_pred)))

The training R_square is: 0.85
The MAE is:19
MSE is: 1623
The RMSE is: 40


In [10]:
y_test_pred = stacking_regressor.predict(X_test)
print('The testing R_square is: %.2f'% r2_score(y_test, y_test_pred))
print('The MAE is:%2.f'%mean_absolute_error(y_test, y_test_pred))
print('MSE is: %2.f'%mean_squared_error(y_test, y_test_pred))
print('The RMSE is: %2.f'% np.sqrt(mean_squared_error(y_test, y_test_pred)))

The testing R_square is: 0.48
The MAE is:38
MSE is: 5772
The RMSE is: 76


In [11]:
#Test Train Split and Standardization data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

In [12]:
estimators = [
    ('knn', KNeighborsRegressor()),
    ('xgb', XGBRegressor(random_state=42)),
    ('rf', RandomForestRegressor(random_state=42))
]

In [13]:
meta_model = RandomForestRegressor(random_state=42)

In [14]:
stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_model
)

In [15]:
stacking_regressor.fit(X_train_std, y_train)

StackingRegressor(estimators=[('knn', KNeighborsRegressor()),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interactio...
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=42, ...)),
                              ('rf', RandomForestRegressor(random_state=42))],
                  final_estimator=RandomForestRegressor(random_state=42))

In [16]:
y_train_pred_std = stacking_regressor.predict(X_train_std)
print('The training R_square is: %.2f'% r2_score(y_train, y_train_pred_std))
print('The MAE is:%2.f'%mean_absolute_error(y_train, y_train_pred_std))
print('MSE is: %2.f'%mean_squared_error(y_train, y_train_pred_std))
print('The RMSE is: %2.f'% np.sqrt(mean_squared_error(y_train, y_train_pred_std)))

The training R_square is: 0.83
The MAE is:19
MSE is: 1783
The RMSE is: 42


In [17]:
y_test_pred_std = stacking_regressor.predict(X_test_std)
print('The testing R_square is: %.2f'% r2_score(y_test, y_test_pred_std))
print('The MAE is:%2.f'%mean_absolute_error(y_test, y_test_pred_std))
print('MSE is: %2.f'%mean_squared_error(y_test, y_test_pred_std))
print('The RMSE is: %2.f'% np.sqrt(mean_squared_error(y_test, y_test_pred_std)))

The testing R_square is: 0.49
The MAE is:37
MSE is: 5671
The RMSE is: 75


In [18]:
estimators = [
    ('knn', KNeighborsRegressor()),
    ('xgb', XGBRegressor(random_state=42)),
    ('rf', RandomForestRegressor(random_state=42))
]

In [19]:
meta_model = XGBRegressor(random_state=42)

In [20]:
stacking_regressor_xgb = StackingRegressor(
    estimators=estimators,
    final_estimator=meta_model
)

In [21]:
stacking_regressor_xgb.fit(X_train_std, y_train)

StackingRegressor(estimators=[('knn', KNeighborsRegressor()),
                              ('xgb',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interactio...
                                               gpu_id=None, grow_policy=None,
                                               importance_type=None,
                                               interaction_constraints=None,
                                               learning_rate=None, max_bin=None,
                                               max_cat_threshold=None,
                                               max_cat_to_onehot=None,
                                               max_delta_step=None,
                                               max_depth=None, max_leaves=None,
                                               min_child_weight=None,
                                               missing=nan,
                                               monotone_constraints=None,
                                               n_estimators=100, n_jobs=None,
                                               num_parallel_tree=None,
                                               predictor=None, random_state=42, ...))

In [22]:
y_train_pred_std_xgb = stacking_regressor_xgb.predict(X_train_std)
print('The training R_square is: %.2f'% r2_score(y_train, y_train_pred_std_xgb))
print('The MAE is:%2.f'%mean_absolute_error(y_train, y_train_pred_std_xgb))
print('MSE is: %2.f'%mean_squared_error(y_train, y_train_pred_std_xgb))
print('The RMSE is: %2.f'% np.sqrt(mean_squared_error(y_train, y_train_pred_std_xgb)))

The training R_square is: 0.79
The MAE is:20
MSE is: 2242
The RMSE is: 47


In [23]:
y_test_pred_std_xgb = stacking_regressor_xgb.predict(X_test_std)
print('The testing R_square is: %.2f'% r2_score(y_test, y_test_pred_std_xgb))
print('The MAE is:%2.f'%mean_absolute_error(y_test, y_test_pred_std_xgb))
print('MSE is: %2.f'%mean_squared_error(y_test, y_test_pred_std_xgb))
print('The RMSE is: %2.f'% np.sqrt(mean_squared_error(y_test, y_test_pred_std_xgb)))

The testing R_square is: 0.45
The MAE is:37
MSE is: 6117
The RMSE is: 78
